# Fauna

This is a deconstruction of parts of fauna. 

* https://github.com/nextstrain/fauna

Scripts:

**zika_upload.py**

```
python3 vdb/zika_upload.py \
  -db vdb \
  -v zika \
  --source genbank \
  --locus genome \
  --fname GenomicFastaResults.fasta
```

**zika_update.py**

```
python3 vdb/zika_update.py \
  -db vdb \
  -v zika \
  --update_citations
```

*check dependencies listed in requirements.txt*

In [1]:
# ==== Packages
import os, re, time, csv, sys
from Bio import SeqIO
from typing import NamedTuple
from datetime import datetime
print("Packages available")

Packages available


## 1. Load an example dataset

Practice on 10 zika sequences. Pull from:

* https://www.viprbrc.org/brc/vipr_genome_search.spg?method=ShowCleanSearch&decorator=flavi_zika

## 2. Process the dataset

In [2]:
# === Input variables
zika_fasta = "../example_data/small.fasta"

# From fauna
strain_fix_fname =  "zika_strain_name_fix.tsv"
location_fix_fname = "zika_location_fix.tsv"
date_fix_fname = "zika_date_fix.tsv"

#virus_fasta_fields = {1:'strain', 3:'collection_date', 4: 'host', 5:'country'}
#sequence_fasta_fields = {0:'accession', 1:'strain'}
# Seems duplicative, replace with:
header_fasta_fields = {0:'accession', 1:'strain', 3:'collection_date', 4: 'host', 5:'country'}
# If we're ignoring anything past 5, then don't pull from vipr

virus="zika" # upload
# ex. 2002-XX-XX or 2002-09-XX
# ex. 2009-06 (Day unknown)
# ex. 2009 (Month and day unknown)
# https://docs.python.org/2/library/datetime.html#strftime-and-strptime-behavior
expected_date_formats = {'%Y_%m_%d','%Y (Month and day unknown)', '%Y-%m (Day unknown)', 
                         '%Y %b %d','%Y-XX-XX', '%Y-%m-%d', '%Y-%m-%dT%H:%M:%SZ'}

**Functions**

In [3]:
# vdl/uploads.py
def define_fixes_dict(fname:str) -> dict[str,str]:
    '''
    Open strain/location/date fixing tsv files and define corresponding dictionaries
    '''
    reader = csv.DictReader(filter(lambda row: row[0]!='#', open(fname)), delimiter='\t')
    fixes_dict = {}
    for line in reader:
        fixes_dict[line['label'].encode().decode('unicode-escape')] = line['fix']
    return fixes_dict

def fixes_str(original_str:str, fixes_dict:dict[str,str]={}):
    '''
    return the new strain name that will replace the original string
    cannot be applied to location and date since key is based on strain name
    '''
    # labmda x: fixes[original_str] if original_str in fixes dict else original_str
    if original_str in fixes_dict:
        return fixes[original_str] 
    else:
        return original_str

# vdl/zika_uploads.py
def fixes_strain_name(name, fixes_dict={}, fixes_tsv:str="") -> (str,str): # Since we can't decide if we want strain or name
    fixes_dict = {}
    if(len(fixes_dict)<1 and len(fixes_tsv)>0):
        fixes_dict = define_fixes_dict(fixes_tsv)
    
    # https://stackoverflow.com/questions/2484156/is-str-replace-replace-ad-nauseam-a-standard-idiom-in-python
    original_name = name
    name = fixes_str(original_name, fixes_dict) 
    name = name.replace('Zika_virus', '').replace('Zikavirus', '').replace('Zika virus', '').replace('Zika', '').replace('ZIKV', '')
    name = name.replace('Human', '').replace('human', '').replace('H.sapiens_wt', '').replace('H.sapiens_tc', '').replace('Hsapiens_tc', '').replace('H.sapiens-tc', '').replace('Homo_sapiens', '').replace('Homo sapiens', '').replace('Hsapiens', '').replace('H.sapiens', '')
    name = name.replace('/Hu/', '')
    name = name.replace('_Asian', '').replace('_Asia', '').replace('_asian', '').replace('_asia', '')
    name = name.replace('_URI', '').replace('_SER', '').replace('_PLA', '').replace('_MOS', '').replace('_SAL', '')
    name = name.replace('Aaegypti_wt', 'Aedes_aegypti').replace('Aedessp', 'Aedes_sp')
    name = name.replace(' ', '').replace('\'', '').replace('(', '').replace(')', '').replace('//', '/').replace('__', '_').replace('.', '').replace(',', '')
    name = re.sub('^[\/\_\-]', '', name)
    try: # ID must start with letter
        name = 'V' + str(int(name))
    except:
        pass
    name = fixes_str(name, fixes_dict)
    return name

# # vdl/parse.py Load data
# def parse_fasta_file(fasta, virus_fasta_fields, sequence_fasta_fields, **kwargs):
#     '''
#     Parse FASTA file with default header formatting
#     :return: list of documents(dictionaries of attributes) to upload
#     '''
#     header_fixes = False
#     if (kwargs["fasta_header_fix"]):
#         header_fixes = {}
#         try:
#             with open(kwargs["fasta_header_fix"], 'rU') as fh:
#                 for line in fh:
#                     if not line.startswith('#'):
#                         k, v = line.strip().split("\t")
#                         header_fixes[k] = v                
#         except IOError:
#             raise Exception(kwargs["fasta_header_fix"], "not found")
#     viruses = []
#     sequences = []
#     try:
#         handle = open(fasta, 'r')
#     except IOError:
#         raise Exception(fasta, "not found")
#     else:
#         for record in SeqIO.parse(handle, "fasta"):
#             if header_fixes:
#                 try:
#                     record.description = header_fixes[record.description]
#                 except KeyError:
#                     raise Exception(record.description, "not in header fix file. Fatal.")
#             content = list(map(lambda x: x.strip(), record.description.replace(">", "").split('|')))
#             v = {key: content[ii] if ii < len(content) else "" for ii, key in virus_fasta_fields.items()}
#             s = {key: content[ii] if ii < len(content) else "" for ii, key in sequence_fasta_fields.items()}
#             s['sequence'] = str(record.seq).lower()
#             #v = self.add_virus_fields(v, **kwargs)
#             #s = self.add_sequence_fields(s, **kwargs)
#             sequences.append(s)
#             viruses.append(v)
#         handle.close()
#     return (viruses, sequences)

# === Only fix casing on the Host?
def fix_casing(self, document): # JC
    for field in ['host']:       # Looping over one entry...hmmmmmmm
        if field in document and document[field] is not None:
            document[field] = self.camelcase_to_snakecase(document[field])

# ===== Main Method
fix_name_dict = define_fixes_dict(strain_fix_fname)  # tsv file in Input
fix_location_dict = define_fixes_dict(location_fix_fname)
fix_date_dict = define_fixes_dict(date_fix_fname)

type(fix_name_dict)
type(fix_location_dict)
type(fix_date_dict)

dict

In [4]:
def format_date(date_str):
    '''
    Format viruses date attribute: collection date in YYYY-MM-DD format, for example, 2016-02-28
    Input date could be YYYY_MM_DD, reformat to YYYY-MM-DD
    '''
    # # ex. 2002_04_25 to 2002-04-25
    # date_fields = []
    # for f in ['date', 'collection_date', 'submission_date']: # <= This is out of scope of responsibilities
    #     if f in virus:
    #         date_fields.append(f)

    #for field in date_fields: # No...
    
    # If date_str is empty, return None
    if date_str is None or date_str.strip() == '':
        date_str = None
        return
        
    date_str = re.sub(r'_', r'-', date_str)
    # ex. 2002-XX-XX or 2002-09-05
    if re.match(r'\d\d\d\d-(\d\d|XX)-(\d\d|XX)', date_str):
        pass
    # ex. 2002-2-4
    elif re.match(r'^\d\d\d\d-\d-\d$', date_str):
        date_str = re.sub(
            r'^(\d\d\d\d)-(\d)-(\d)$', r'\1-0\2-0\3', date_str)
    # ex. 2002-02-4
    elif re.match(r'^\d\d\d\d-\d\d-\d$', date_str):
        date_str = re.sub(
            r'^(\d\d\d\d)-(\d\d)-(\d)$', r'\1-\2-0\3', date_str)
    # ex. 2002-2-15
    elif re.match(r'^\d\d\d\d-\d-\d\d$', date_str):
        date_str = re.sub(
            r'^(\d\d\d\d)-(\d)-(\d\d)$', r'\1-0\2-\3', date_str)
    elif re.match(r'\d\d\d\d\s\(Month\sand\sday\sunknown\)', date_str):
        date_str = date_str[0:4] + "-XX-XX"
    # ex. 2009-06 (Day unknown)
    elif re.match(r'\d\d\d\d-\d\d\s\(Day\sunknown\)', date_str):
        date_str = date_str[0:7] + "-XX"
    elif re.match(r'\d\d\d\d-\d\d', date_str):
        date_str = date_str[0:7] + "-XX"
    elif re.match(r'\d\d\d\d', date_str):
        date_str = date_str[0:4] + "-XX-XX"
    else:
        print("Couldn't reformat this date: " +
              date_str + ", setting to None")
        date_str = None
    return date_str

print(format_date("2002-XX-XX"))
print(format_date("2002-09-05"))
print(format_date("2002-09"))
print(format_date("2002-2-4"))
print(format_date("2002-02-4"))
print(format_date("2002-02-4"))
print(format_date("2002-02-4"))
print(format_date("2002-02-4"))


2002-XX-XX
2002-09-05
2002-09-XX
2002-02-04
2002-02-04
2002-02-04
2002-02-04
2002-02-04


In [5]:
def ncov_ingest_format_date(date_string: str, expected_formats: set) -> str:
    """
    Format *date_string* to ISO 8601 date (YYYY-MM-DD).
    If *date_string* does not match *expected_formats*, return *date_string*.
    >>> expected_formats = {'%Y-%m-%d', '%Y-%m-%dT%H:%M:%SZ'}
    >>> format_date("2020", expected_formats)
    '2020'
    >>> format_date("2020-01", expected_formats)
    '2020-01'
    >>> format_date("2020-1-15", expected_formats)
    '2020-01-15'
    >>> format_date("2020-1-1", expected_formats)
    '2020-01-01'
    >>> format_date("2020-01-15", expected_formats)
    '2020-01-15'
    >>> format_date("2020-01-15T00:00:00Z", expected_formats)
    '2020-01-15'
    >>> format_date("2020-XX-XX", expected_formats)
    '2020'
    >>> format_date("2020 (Month and day unknown)", expected_formats)
    '2020'
    >>> format_date("2020-06 (Day unknown)", expected_formats)
    '2020-06'
    """
    
    
    
    for date_format in expected_formats:
        try:
            date_string = date_string.replace(" .*","")
            return datetime.strptime(date_string, date_format).strftime('%Y-%m-%d')
        except ValueError:
            continue

    return date_string

# ex. 2002-XX-XX or 2002-09-XX
# ex. 2009-06 (Day unknown)
# ex. 2009 (Month and day unknown)
# https://docs.python.org/2/library/datetime.html#strftime-and-strptime-behavior
expected_date_formats = {'%Y_%m_%d','%Y (Month and day unknown)', '%Y-%m (Day unknown)', 
                         '%Y %b %d','%Y-XX-XX', '%Y-%m-%d', '%Y-%m-%dT%H:%M:%SZ'}

print(ncov_ingest_format_date("2020_08_23", expected_date_formats))
print(ncov_ingest_format_date("2020-8-24", expected_date_formats))
print(ncov_ingest_format_date("2020-8-1", expected_date_formats))
print(ncov_ingest_format_date("2020-09", expected_date_formats))
print(ncov_ingest_format_date("2020-XX-XX", expected_date_formats))  # resets to Jan 1st
print(ncov_ingest_format_date("2020 (Month and day unknown)", expected_date_formats))
print(ncov_ingest_format_date("2020-10 (Day unknown)", expected_date_formats)) # resets day to 1st
print(ncov_ingest_format_date("2020 Aug 23", expected_date_formats))
print(ncov_ingest_format_date("2020-XX-1X (hi", expected_date_formats))
print(ncov_ingest_format_date("2020 (hi", expected_date_formats))

# Catch up on ambiguous dates thread
# Hmm: https://github.com/nextstrain/augur/blob/master/augur/util_support/date_disambiguator.py
# https://github.com/nextstrain/augur/issues/602
# https://github.com/nextstrain/augur/pull/631

2020-08-23
2020-08-24
2020-08-01
2020-09
2020-01-01
2020-01-01
2020-10-01
2020-08-23
2020-XX-1X (hi
2020 (hi


Use Augur's date.py

```
git clone https://github.com/victorlin/augur.git
cd augur
git checkout filter/sqlite
less augur/dates.py
```

In [6]:
import argparse
#import datetime
import re
import isodate
import treetime
from datetime import date
from textwrap import dedent
from functools import lru_cache
from typing import Any, List


SUPPORTED_DATE_HELP_TEXT = dedent("""\
    1. an Augur-style numeric date with the year as the integer part (e.g. 2020.42) or
    2. a date in ISO 8601 date format (i.e. YYYY-MM-DD) (e.g. '2020-06-04') or
    3. a backwards-looking relative date in ISO 8601 duration format with optional P prefix (e.g. '1W', 'P1W')
""")


class InvalidDateFormat(ValueError):
    pass


# float, negative ok
# note: year-only is treated as incomplete ambiguous and must be non-negative (see RE_YEAR_ONLY)
RE_NUMERIC_DATE = re.compile(r'^-*\d+\.\d+$')

# complete ISO 8601 date
# e.g. 2018-03-25
RE_ISO_8601_DATE = re.compile(r'^\d{4}-\d{2}-\d{2}$')

# complete ambiguous ISO 8601 date
# e.g. 2018-03-XX
RE_AMBIGUOUS_ISO_8601_DATE = re.compile(r'^[\dX]{4}-[\dX]{2}-[\dX]{2}$')

# incomplete ambiguous ISO 8601 date (missing day)
# e.g. 2018-03
RE_AMBIGUOUS_ISO_8601_DATE_YEAR_MONTH = re.compile(r'^[\dX]{4}-[\dX]{2}$')
#RE_AMBIGUOUS_ISO_8601_DATE_YEAR_MONTH = re.compile(r'^[\dX]{4}-[\dX]{2}\s')

# incomplete ambiguous ISO 8601 date (missing month and day)
# e.g. 2018
# and other non-negative ints
# e.g. 0, 1, 123, 12345
RE_YEAR_ONLY = re.compile(r'^[\dX]+$')

# also support relative dates (ISO 8601 durations) - see any_to_numeric()


CACHE_SIZE = 8192
# Some functions below use @lru_cache to minimize redundant operations on
# large datasets that are likely to have multiple entries with the same date value.


@lru_cache(maxsize=CACHE_SIZE)
def get_year(date_in:Any):
    """Get the year from a date. Only works for ISO dates.

    This function is intended to be registered as a user-defined function in sqlite3.
    """
    date_in = str(date_in)
    try:
        return int(date_in.split('-')[0])
    except (ValueError, IndexError):
        return None


@lru_cache(maxsize=CACHE_SIZE)
def get_month(date_in:Any):
    """Get the month from a date. Only works for ISO dates.

    This function is intended to be registered as a user-defined function in sqlite3.
    """
    date_in = str(date_in)
    try:
        return int(date_in.split('-')[1])
    except (ValueError, IndexError):
        return None


@lru_cache(maxsize=CACHE_SIZE)
def get_day(date_in:Any):
    """Get the day from a date. Only works for ISO dates.

    This function is intended to be registered as a user-defined function in sqlite3.
    """
    date_in = str(date_in)
    try:
        return int(date_in.split('-')[2])
    except (ValueError, IndexError):
        return None


def iso_to_numeric(date_in:str, ambiguity_resolver:str):
    """Convert ISO 8601 date string to numeric, resolving any ambiguity detected by explicit 'X' characters or missing date parts."""
    date_parts = date_in.split('-', maxsplit=2)
    # TODO: resolve partial month/day ambiguity eg. 2018-1X-XX, 2018-10-3X
    if ambiguity_resolver == 'min':
        year = int(date_parts[0].replace('X', '0'))
        month = int(date_parts[1]) if len(date_parts) > 1 and date_parts[1].isnumeric() else 1
        day = int(date_parts[2]) if len(date_parts) > 2 and date_parts[2].isnumeric() else 1
        return date_to_numeric_capped(date(year, month, day))
    if ambiguity_resolver == 'max':
        year = int(date_parts[0].replace('X', '9'))
        month = int(date_parts[1]) if len(date_parts) > 1 and date_parts[1].isnumeric() else 12
        if len(date_parts) == 3 and date_parts[2].isnumeric():
            day = int(date_parts[2])
        else:
            if month in {1,3,5,7,8,10,12}:
                day = 31
            elif month == 2:
                day = 28
            else:
                day = 30
        return date_to_numeric_capped(date(year, month, day))


def any_to_numeric(date_in:Any, ambiguity_resolver:str):
    """Return numeric date if date is in a supported format.

    For ambiguous ISO 8601 dates, resolve to either minimum or maximum possible value.
    """
    date_in = str(date_in)

    # Absolute date in numeric format.
    if RE_NUMERIC_DATE.match(date_in):
        return float(date_in)

    # Absolute date in potentially incomplete/ambiguous ISO 8601 date format.
    if (RE_ISO_8601_DATE.match(date_in) or
        RE_AMBIGUOUS_ISO_8601_DATE.match(date_in) or
        RE_AMBIGUOUS_ISO_8601_DATE_YEAR_MONTH.match(date_in) or
        RE_YEAR_ONLY.match(date_in)
        ):
        return iso_to_numeric(date_in, ambiguity_resolver)

    # Relative date in ISO 8601 duration format.
    # No regex for this (it is complex), just try evaluating last and
    # let any expected errors pass to raise the general-purpose InvalidDateFormat.
    try:
        # make a copy of date_in for this block
        duration_str = str(date_in)
        if not duration_str.startswith('P'):
            duration_str = 'P'+duration_str
        return treetime.utils.numeric_date(datetime.date.today() - isodate.parse_duration(duration_str))
    except (ValueError, isodate.ISO8601Error):
        pass

    raise InvalidDateFormat(f"""Unable to determine date from '{date_in}'. Ensure it is in one of the supported formats:\n{SUPPORTED_DATE_HELP_TEXT}""")



def any_to_numeric_type_min(date_in:Any):
    """Get the numeric date from any supported date format, taking the minimum possible value if ambiguous.

    This function is intended to be used as the `type` parameter in `argparse.ArgumentParser.add_argument()`

    This raises an ArgumentTypeError from InvalidDateFormat exceptions, otherwise the custom exception message won't be shown in console output due to:
    https://github.com/python/cpython/blob/5c4d1f6e0e192653560ae2941a6677fbf4fbd1f2/Lib/argparse.py#L2503-L2513
    """
    try:
        return any_to_numeric(date_in, ambiguity_resolver='min')
    except InvalidDateFormat as e:
        raise argparse.ArgumentTypeError(str(e)) from e


def any_to_numeric_type_max(date_in:Any):
    """Get the numeric date from any supported date format, taking the maximum possible value if ambiguous.

    This function is intended to be used as the `type` parameter in `argparse.ArgumentParser.add_argument()`

    This raises an ArgumentTypeError from InvalidDateFormat exceptions, otherwise the custom exception message won't be shown in console output due to:
    https://github.com/python/cpython/blob/5c4d1f6e0e192653560ae2941a6677fbf4fbd1f2/Lib/argparse.py#L2503-L2513
    """
    try:
        return any_to_numeric(date_in, ambiguity_resolver='max')
    except InvalidDateFormat as e:
        raise argparse.ArgumentTypeError(str(e)) from e


@lru_cache(maxsize=CACHE_SIZE)
def try_get_numeric_date_min(date_in:Any):
    """Get the numeric date from any supported date format, taking the minimum possible value if ambiguous.

    This function is intended to be registered as a user-defined function in sqlite3.
    """
    try:
        return any_to_numeric(date_in, ambiguity_resolver='min')
    except ValueError:
        return None


@lru_cache(maxsize=CACHE_SIZE)
def try_get_numeric_date_max(date_in:Any):
    """Get the numeric date from any supported date format, taking the maximum possible value if ambiguous.

    This function is intended to be registered as a user-defined function in sqlite3.
    """
    try:
        return any_to_numeric(date_in, ambiguity_resolver='max')
    except ValueError:
        return None


@lru_cache(maxsize=CACHE_SIZE)
def get_date_errors(date_in:Any):
    """Check date for any errors.

    assert_only_less_significant_ambiguity:

    If an exception is raised here, it will result in a `sqlite3.OperationalError`
    without trace to the original exception. For this reason, if the check raises
    :class:`InvalidDateFormat`, return a constant string
    `ASSERT_ONLY_LESS_SIGNIFICANT_AMBIGUITY_VALUE` which sqlite3 can then "handle".

    This function is intended to be registered as a user-defined function in sqlite3.
    """
    date_in = str(date_in)
    if not date_in:
        # let empty string pass silently
        return None
    if RE_NUMERIC_DATE.match(date_in):
        # let numeric dates pass silently
        return None
    if date_in[0] == '-':
        # let negative ISO dates pass silently
        return None
    date_parts = date_in.split('-', maxsplit=2)
    try:
        assert_only_less_significant_ambiguity(date_parts)
    except InvalidDateFormat as e:
        return str(e)


ASSERT_ONLY_LESS_SIGNIFICANT_AMBIGUITY_ERROR = 'assert_only_less_significant_ambiguity'


def assert_only_less_significant_ambiguity(date_parts:List[str]):
    """
    Raise an exception if a constrained digit appears in a less-significant place
    than an uncertain digit.

    These patterns are valid:
        2000-01-01
        2000-01-XX
        2000-XX-XX

    but this is invalid, because month is uncertain but day is constrained:
        2000-XX-01

    These invalid cases are assumed to be unintended use of the tool.
    """
    has_exact_year = date_parts[0].isnumeric()
    has_exact_month = len(date_parts) > 1 and date_parts[1].isnumeric()
    has_exact_day = len(date_parts) > 2 and date_parts[2].isnumeric()
    if has_exact_day and not (has_exact_month and has_exact_year):
        raise InvalidDateFormat(ASSERT_ONLY_LESS_SIGNIFICANT_AMBIGUITY_ERROR)
    if has_exact_month and not has_exact_year:
        raise InvalidDateFormat(ASSERT_ONLY_LESS_SIGNIFICANT_AMBIGUITY_ERROR)



### date_to_numeric logic ###
# copied from treetime.utils.numeric_date
# simplified+cached for speed

from calendar import isleap
def date_to_numeric(d:date):
    """Return the numeric date representation of a datetime.date."""
    days_in_year = 366 if isleap(d.year) else 365
    return d.year + (d.timetuple().tm_yday-0.5) / days_in_year


@lru_cache(maxsize=CACHE_SIZE)
def date_to_numeric_capped(d:date, max_numeric:float=date_to_numeric(date.today())):
    """Return the numeric date representation of a datetime.date, capped at a maximum numeric value."""
    d_numeric = date_to_numeric(d)
    if d_numeric > max_numeric:
        d_numeric = max_numeric
    return d_numeric


## 3. Upload to fauna (nope)

## 3. BioPython reorg

In [7]:
fname=zika_fasta

# Early exit if file not found
try:
    fhandle = open(fname, 'r')
except IOError:
    raise Exception(fname, "not found")

fix_name_dict = define_fixes_dict(strain_fix_fname) 
fix_location_dict = define_fixes_dict(location_fix_fname)
fix_date_dict = define_fixes_dict(date_fix_fname)
    
try:
    shandle = open("sequences.fasta", 'w')
    mhandle = open("metadata.tsv", 'w')
    shandle.close()
    mhandle.close()
    shandle = open("sequences.fasta", 'a')
    mhandle = open("metadata.tsv", 'a')
    mhandle.write("\t".join(("strain", "virus", "accession", "date", "region", "country", "division", "city", "db", "segment", "authors", "url", "title", "journal", "paper_url"))+"\n")
except IOError:
    raise Exception('Cannot write to sequences.fasta and/or metadata.tsv')

for record in SeqIO.parse(fhandle, "fasta"):
    #print(record.id) # Header, Breaks at spaces!
    print(record.description) # Whole header
    content = list(
        map(lambda x: x.strip(), 
            record.description
            .replace(" ", "_") # Deal with spaces
            .split('|'))
    )
    print(content)
    metadata = {key: content[ii] if ii < len(content) else "" for ii, key in header_fasta_fields.items()}
    print("metadata=", metadata)
    metadata["strain"] = fixes_strain_name(metadata["strain"], fixes_dict=fix_name_dict)
    # metadata["collection_date"] = fixes_str(metadata["strain"], fix_date_dict) # based on fixed strain name?
    # metadata["location"] = fixes_str(metadata["strain"], fix_location_dict)  # find where location is defined
    print("metadata[strain]=", metadata["strain"])
    
    # Hmm, was an obj method (checking for "date", "collection date", "submission date", seems too specialized...)
    # If you want to check for all dates, then check all fields for a XXXX-XX-XX or similar date format...
    
    metadata["collection_date"]=ncov_ingest_format_date(metadata["collection_date"], expected_date_formats)
    print("metadata[collection_date]=", metadata["collection_date"])
    
    # Stream a fasta file, do not read all into memory
    # Append to metadata file
    # Append to sequence file
    shandle.write(">" + metadata["strain"] + "\n")
    shandle.write(str(record.seq) + "\n") # split in lines of 80 char
    mhandle.write("\t".join((metadata["strain"], "zika", metadata["accession"], metadata["collection_date"]))+"\n")

shandle.close()
mhandle.close()
fhandle.close()
%whos

KY241742|ZIKV_SG_072|NA|2016_08_28|Human|Singapore|Asian|Zika_virus
['KY241742', 'ZIKV_SG_072', 'NA', '2016_08_28', 'Human', 'Singapore', 'Asian', 'Zika_virus']
metadata= {'accession': 'KY241742', 'strain': 'ZIKV_SG_072', 'collection_date': '2016_08_28', 'host': 'Human', 'country': 'Singapore'}
metadata[strain]= SG_072
metadata[collection_date]= 2016-08-28
MF098771|Mexico_Rus_12TVR_2017|NA|2017_01_30|Human|Russia|Asian|Zika_virus
['MF098771', 'Mexico_Rus_12TVR_2017', 'NA', '2017_01_30', 'Human', 'Russia', 'Asian', 'Zika_virus']
metadata= {'accession': 'MF098771', 'strain': 'Mexico_Rus_12TVR_2017', 'collection_date': '2017_01_30', 'host': 'Human', 'country': 'Russia'}
metadata[strain]= Mexico_Rus_12TVR_2017
metadata[collection_date]= 2017-01-30
MF098768|Dominican_Rep_Rus_7EGR_2016|NA|2016_08_25|Human|Russia|Asian|Zika_virus
['MF098768', 'Dominican_Rep_Rus_7EGR_2016', 'NA', '2016_08_25', 'Human', 'Russia', 'Asian', 'Zika_virus']
metadata= {'accession': 'MF098768', 'strain': 'Dominican_Re

# Aiming at a sequence and metadata file

**sequence.fasta**

Only strain name in header

```
>PAN/CDC_259359_V1_V3/2015
gaatttgaagcgaatgctaacaacagtatcaacaggttttattttggatttggaaacgag
agtttctggtcatgaaaaacccaaaaaagaaatccggaggattccggattgtcaatatgc
...
```

**metadata.tsv**

```
strain  virus   accession       date    region  country division        city    db      segment authors url     title   journal paper_url
PAN/CDC_259359_V1_V3/2015       zika    KX156774        2015-12-18      North America   Panama  Panama  Panama  genbank genome  Shabman et al   https://www.ncbi.nlm.nih.gov/nuccore/KX156774       Direct Submission       Submitted (29-APR-2016) J. Craig Venter Institute, 9704 Medical Center Drive, Rockville, MD 20850, USA  https://www.ncbi.nlm.nih.gov/pubmed/
...
```

which can also look like:

```
strain=PAN/CDC_259359_V1_V3/2015=
virus=zika
accession=KX156774
date=2015-12-18
region=North America
country=Panama
division=Panama
city=Panama
db=genbank
segment=genome
authors=Shabman et al
url=https://www.ncbi.nlm.nih.gov/nuccore/KX156774
title=Direct Submission
journal=Submitted (29-APR-2016) J. Craig Venter Institute, 9704 Medical Center Drive, Rockville, MD 20850, USA
paper_url=https://www.ncbi.nlm.nih.gov/pubmed/
```

### Zika Update (genbank sections)

In [8]:
# def update_citations(self, database, table, preview, index='accession', **kwargs):
#   print("Updating citation fields")
#   _, sequences = self.get_genbank_sequences(**kwargs)
#   citation_keys = ['authors', 'title', 'journal', 'puburl', 'url', index]
#   sequences = [{key: doc[key] for key in citation_keys} for doc in sequences]
#   if not preview:
#     print("Updating " + str(len(sequences)) + " sequence citations in " + database + "." + table)
#     self.upload_to_rethinkdb(database, table, sequences, overwrite=True, index='accession')
#   else:
#     print("Preview of updates to be made, remove --preview to make updates to database")

# def get_genbank_sequences(self, email, **kwargs):
#     if self.accessions is None:
#         table = self.virus + "_sequences"
#         accessions = self.get_accessions(self.database, table)
#     else:
#         accessions = [acc.strip() for acc in self.accessions.split(",")]
#     self.entrez_email(email)
#     gi = self.get_GIs(accessions, kwargs["n_entrez"])
#     return self.get_entrez_viruses(gi, **kwargs)

# def get_accessions(self, database, table):
#     '''
#     Return documents from the table.database for which accession numbers are from genbank
#     '''
#     print("Getting accession numbers for sequences obtained from Genbank")
#     accessions = list(r.db(database).table(table).filter((r.row["source"] == 'genbank') | (r.row["source"] == 'vipr')).get_field('accession').run())
#     return accessions

# def update_locations(self, database, table, preview, **kwargs):
#     print("Updating location fields")
#     viruses = list(r.table(table).run())
#     self.define_countries("source-data/geo_synonyms.tsv")
#     self.define_regions("source-data/geo_regions.tsv")
#     viruses = self.reassign_new_locations(viruses, self.location_fields, **kwargs)
#     if not preview:
#         print("Updating " + str(len(viruses)) + " virus locations in " + self.database + "." + self.viruses_table)
#         del kwargs['overwrite']
#         self.upload_to_rethinkdb(database, table, viruses, overwrite=True, index='strain')
#     else:
#         print("Preview of updates to be made, remove --preview to make updates to database")

# Genbank Entrez

In [11]:
from Bio import Entrez
Entrez.email = "Your.Name.Here@example.org"
handle = Entrez.einfo() # or esearch, efetch, ...
record = Entrez.read(handle)
print(record)
handle.close()

# Hmm fetch in batches, which won't work if we're outputing the metadata one record at a time, unless we batch the metadata output.


#     def get_entrez_viruses(self, giList, **kwargs):
#         '''
#         Use entrez efetch to get genbank entries from genbank identifiers
#         '''
#         ## define batch size for download
#         batchSize = 100

#         # post NCBI query
#         try:
#             search_handle = Entrez.epost(db=self.gbdb, id=",".join(giList))
#             search_results = Entrez.read(search_handle)
#             webenv, query_key = search_results["WebEnv"], search_results["QueryKey"]
#         except:
#             print("ERROR: Couldn't connect with entrez, please run again")

#         viruses = []
#         sequences = []
#         #fetch all results in batch of batchSize entries at once
#         for start in range(0,len(giList),batchSize):
#             #fetch entries in batch
#             try:
#                 handle = Entrez.efetch(db=self.gbdb, rettype="gb", retstart=start, retmax=batchSize, webenv=webenv, query_key=query_key)
#             except IOError:
#                 print("ERROR: Couldn't connect with entrez, please run again")
#             else:
#                 result = self.parse_gb_entries(handle, **kwargs)
#                 viruses.extend(result[0])
#                 sequences.extend(result[1])
#         return (viruses, sequences)

{'DbList': ['pubmed', 'protein', 'nuccore', 'ipg', 'nucleotide', 'structure', 'genome', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'gap', 'gapplus', 'grasp', 'dbvar', 'gene', 'gds', 'geoprofiles', 'homologene', 'medgen', 'mesh', 'ncbisearch', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'popset', 'proteinclusters', 'pcassay', 'protfam', 'pccompound', 'pcsubstance', 'seqannot', 'snp', 'sra', 'taxonomy', 'biocollections', 'gtr']}
